In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from cvxpy import *
import sys
import os
print(os.getcwd())

ModuleNotFoundError: No module named 'cvxpy'

In [62]:
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
origin_df = pd.read_csv(url)
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       7214 non-null   int64  
 1   name                     7214 non-null   object 
 2   first                    7214 non-null   object 
 3   last                     7214 non-null   object 
 4   compas_screening_date    7214 non-null   object 
 5   sex                      7214 non-null   object 
 6   dob                      7214 non-null   object 
 7   age                      7214 non-null   int64  
 8   age_cat                  7214 non-null   object 
 9   race                     7214 non-null   object 
 10  juv_fel_count            7214 non-null   int64  
 11  decile_score             7214 non-null   int64  
 12  juv_misd_count           7214 non-null   int64  
 13  juv_other_count          7214 non-null   int64  
 14  priors_count            

In [63]:
df = origin_df.loc[:,["age_cat","sex","race","priors_count","c_charge_degree","two_year_recid"]].query('race in ["African-American","Caucasian"]')
df.reset_index(drop=True,inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6150 entries, 0 to 6149
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age_cat          6150 non-null   object
 1   sex              6150 non-null   object
 2   race             6150 non-null   object
 3   priors_count     6150 non-null   int64 
 4   c_charge_degree  6150 non-null   object
 5   two_year_recid   6150 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 288.4+ KB


In [161]:
features = ["age_cat", "race", "sex", "priors_count", "c_charge_degree"]
cont_features = ["priors_count"]
predicted_feature = "two_year_recid"
sensitive_feature = "race"

In [174]:
def processing(df):
    df = df.dropna(subset=["days_b_screening_arrest"]).loc[:,fearures + predicted_feature].query('race in ["African-American","Caucasian"]')

    data = df.to_dict('list')
    for k in data.keys():
        data[k] = np.array(data[k])

    y = data[predicted_feature]
    y[y==0] = -1


    X = np.array([]).reshape(len(y), 0) # empty array with num rows same as num examples, will hstack the features to it
    x_sensitive_feature = None

    feature_names = []
    for feature in features:
        vals = data[feature]
        if feature in cont_features:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals) 
            vals = np.reshape(vals, (len(y), -1)) 

        else: 
            lb = preprocessing.LabelBinarizer()
            lb.fit(vals)
            vals = lb.transform(vals)

        if feature == sensitive_feature:
            x_sensitive_feature = vals


        X = np.hstack((X, vals))

        if feature in cont_features: 
            feature_names.append(attr)
        else: # categorical features
            if vals.shape[1] == 1: 
                feature_names.append(attr)
            else:
                for k in lb.classes_: 
                    feature_names.append(attr + "_" + str(k))

    x_sensitive_feature = np.array(x_sensitive_feature).flatten()
    X = np.concatenate((np.ones(X.shape[0]).reshape(X.shape[0], 1), X), axis = 1)

    feature_names = ["intercept"] + feature_names
    print(f"Features we will be using for classification are: {feature_names}")


    return X, y, x_sensitive_feature

In [171]:
X,y,x_race = prepocessing(origin_df)
print(X,y,x_race)

Features we will be using for classification are:['intercept', 'age_cat_25 - 45', 'age_cat_Greater than 45', 'age_cat_Less than 25', 'race', 'sex', 'priors_count', 'c_charge_degree']
[[ 1.          1.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          0.          0.         ...  1.          0.05593295
   0.        ]
 [ 1.          1.          0.         ...  1.          2.02993903
   0.        ]
 ...
 [ 1.          0.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          0.          0.         ...  1.         -0.73366948
   0.        ]
 [ 1.          1.          0.         ...  0.         -0.14146765
   1.        ]] [ 1  1  1 ... -1 -1 -1] [0 0 1 ... 0 0 0]


In [172]:
X_train,X_test,y_train,y_test,x_race_train,x_race_test = train_test_split(X,y,x_race,test_size=0.5,random_state=5243)


In [170]:
def train_model(X,y,x_sensitive_feature,eps):
    max_iters = 100 
    max_iter_dccp = 50 

    
    num_points, num_features = X.shape
    w = Variable(num_features)

    np.random.seed(112233)
    w.value = np.random.rand(x.shape[1])

(5915, 8)